# This IPython notebook is a document for producing HYSPLIT back trajectories and organizing the files
    by Xia and Butorovic for JGR-Atmospheres

Python version: 3.7
PySPLIT version: 0.3.5

PySPLIT is run under pysplitenv conda virtual environment with all dependencies installed.
See https://github.com/mscross/pysplit for details.

In [ ]:
import pysplit

Running trajectory modeling from GDAS dataset:

In [ ]:
working_dir = r"C:/hysplit4/working"
storage_dir = r"C:/trajectories/PA_GDAS_new"
meteo_dir = r"C:/gdas"
# GDAS meteorological dataset name format is following like "gdas1.apr05.w1"
basename = "PA"
years = list(range(2016,2018))
months = list(range(1,13))
hours = [0,6,12,18]
altitudes = [500,1000,1500,2000]
location = (-53.0, -70.8333)
runtime = -120

pysplit.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime,
                          monthslice = slice(0,32,1), 
                          meteo_bookends = ([4,5],[1]),
                          get_reverse = False,
                          get_clipped = False)

print("GDAS trajectory generation is completed.")



Running trajectory modeling from NCEP/NCAR dataset:

In [ ]:
working_dir = r"C:/hysplit4/working"
storage_dir = r"C:/trajectories/PA_NCAR_new"
meteo_dir = r"C:/NCAR"
# NCEP/NCAR meterological dataset name format is following like "RP.apr2008.gbl1"
# meteoyr_2digits should be set as "False" in pypslit.generate_bulktraj() below
basename = "PA"
years = list(range(1990,2005))
months = list(range(1,13))
hours = [0,6,12,18]
altitudes = [500,1000,1500,2000]
location = (-53.0, -70.8333)
runtime = -120

pysplit.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime,
                          monthslice = slice(0,32,1),
                          meteo_bookends=([1], [1]),
                          get_reverse = False,
                          get_clipped = False,
                          meteoyr_2digits = False)

print("NCEP/NCAR trajectory generation is completed.")

After trajectory generation was finished, trajectory files should be put into subfolders based on their altitudes.
This can be done by searching keywords for file names (on Windows PC) as (for example): "2000s" OR "2000w" OR "2000a".
Otherwise, PySPLIT trajgroup will be confused by file names that contain year 2000 with thoes contain day 20 and hour 00.
I named subfolders as "0500m", "1000m", "1500m", "2000m".

Trajectory files for days without precipitation are deleted:

In [ ]:
import os
import pandas as pd
month_dict1 = (1,2,3,4,5,6,7,8,9,10,11,12)
month_dict2 = ("01","02","03","04","05","06","07","08","09","10","11","12")
month_dict3 = ("jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec")
month_dict4 = ("summer","summer","autumn","autumn","autumn","winter","winter","winter","spring","spring","spring","summer")
month_dict = dict(zip(month_dict1,list(zip(month_dict2,month_dict3,month_dict4))))

def delete_file(data,year,month,day,hour,elevation):
    filename = ("C:/trajectories/PA_{}_new/{}m/PA{}{}{}{}{}{}{}".format(data,elevation,
                                                                 month_dict[month][1],
                                                                 elevation,
                                                                 month_dict[month][2],
                                                                 year,
                                                                 month_dict[month][0],
                                                                 str(day).zfill(2),
                                                                 str(hour).zfill(2)))
    if os.path.exists(filename):
        os.remove(filename)
        print(filename,"is deleted")
    else:
        print(filename,"is not found")

df = pd.read_excel("Airport data/Punta Arenas AP precipitation.xlsx",sheet_name = "6hour")
for i in range(0,len(df)):
    if df["6hr precipitation"][i] == 0 or pd.isnull(df["6hr precipitation"][i]):
        if df["year"][i] in range(2005,2018):
            for elev in ["0500","1000","1500","2000"]:
                delete_file("GDAS",df["year"][i],df["month"][i],df["day"][i],df["hour"][i],elev)
        if df["year"][i] in range(1990,2005):
            for elev in ["0500","1000","1500","2000"]:
                delete_file("NCAR",df["year"][i],df["month"][i],df["day"][i],df["hour"][i],elev)

One trajectory file (elevation = 1000 m) generated has incomplete file (which can be quickly located from file size smaller than 21/22 kb). It was deleted manually.